# rasterio

(bands, rows, columns)

https://rasterio.readthedocs.io/en/latest/topics/index.html !? not in index


? verify crs and coord <-> pixel conversion functions and aviris

open in update mode? r+, w ?


how does rasterio memory use work? load to mem or stay on disk?

- rasterio depends on GDAL
- there is a command line interface `rio`

- https://rasterio.readthedocs.io/en/latest/ (also at https://mapbox.github.io/rasterio/ but maybe older?)
- https://github.com/mapbox/rasterio
- https://github.com/mapbox/rasterio-cookbook
- https://sgillies.net/tags/rasterio.html
- https://github.com/sgillies/affine

see also shapely and fiona, maybe also cartopy, pyproj, pysal, etc.

this looks interesting https://github.com/atedstone/georaster https://georaster.readthedocs.io/en/latest/

todo: read up on this good ref https://chris35wills.github.io/courses/pydata_stack/

note: installing dev version caused a lot of packages to downgrad!

    onda install -c conda-forge/label/dev rasterio

# rasterio changes

https://rasterio.readthedocs.io/en/latest/topics/migrating-to-v1.html



 ## understanding rasterio
 
 Affine?
 
 Rasterio is confusing.
 
 The basic `rasterio` module is in [`__init__.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/__init__.py)

`_base.pyx` [[spource](https://github.com/mapbox/rasterio/blob/master/rasterio/_base.pyx)] defines `DatasetBase`. Some methods include `get_crs()`, `get_transform()`, as well as `read_crs()` and `read_transform()` but they are just called by the corresponding get methods to set the values of `_crs` and `_transform` if they are not already set.

__i think some need to be imported individually!__ e.g. rasterio.plot
and the underscore versions are some compiled thing

https://rasterio.readthedocs.io/en/latest/api/ lists modules and submodules but not all are available, note [old version](https://mapbox.github.io/rasterio/api/) 
 
- `compat`
- `coords`
- crs `_crs`
- drivers `_drivers` 
- `dtypes`
- `enums`
- `env`
- `errors`
- features `_features`
- fill `_fill`
- io `_io`?
- mask ?
- merge ?
- plot ?
- `profiles`
- sample ?
- `transform`
- `vfs`
- vrt ?
- warp `_warp`
- `windows`
 
`rasterio.io` is kind of available as `rasterio._io` but not certain

https://rasterio.readthedocs.io/en/latest/_modules/ is a code browser

then there's this single page thing https://mapbox.github.io/rasterio/lib/rasterio.html which is not in the readthedocs version! wtf!

In [ ]:
import rasterio

In [ ]:
# examine rasterio package
# dir(rasterio)

questions and tasks:

- learn about masks

## opening a raster file

- `rasterio.open()` [[source](https://rasterio.readthedocs.io/en/latest/api/index.html#rasterio.open)] returns a `DatasetReader` [[source](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader)]
- `rasterio.band()` allows you to wrap a dataset and and one or more of its bands up into a `rasterio.Band`, which is really just a tuple:

        Band = namedtuple('Band', ['ds', 'bidx', 'dtype', 'shape'])

definitions and inheritance:

- `io.DatasetReader`, `io.MemoryFile`, etc. are defined in `io.py` [[source](https://github.com/mapbox/rasterio/blob/master/rasterio/io.py)].
- `DataSetReader` inherits from `rasterio._io.DatasetReaderBase`, `rasterio.windows.WindowMethodsMixin`, `rasterio.transform.TransformMethodsMixin`
- `_io.pyx` [[source](https://github.com/mapbox/rasterio/blob/master/rasterio/_io.pyx)] defines `DatasetReaderBase`, `MemoryFileBase`, `InMemoryRaster`, and some writer classes, also includes `read()`

see also MemoryFile? for in-memory?

read https://github.com/mapbox/rasterio/issues/86 sometime!
and see https://mapbox.github.io/rasterio/topics/migrating-to-v1.html

In [ ]:
rasterio.open?

In [ ]:
rasterio.band

In [ ]:
https://mapbox.github.io/rasterio/topics/windowed-rw.html

## reading files into memory?


https://rasterio.readthedocs.io/en/latest/topics/reading.html NOT IN INDEX!

`dataset.read()` defined in `_io.pyx`



## plotting in rasterio

- https://rasterio.readthedocs.io/en/latest/topics/plotting.html [[old](https://mapbox.github.io/rasterio/topics/plotting.html)]
- https://rasterio.readthedocs.io/en/latest/api/rasterio.plot.html [[old](https://mapbox.github.io/rasterio/api/rasterio.plot.html)]

see https://github.com/mapbox/rasterio/blob/master/rasterio/plot.py

`rasterio.plot.show(source, with_bounds=True, contour=False, contour_label_kws=None, ax=None, title=None, transform=None, adjust='linear', **kwargs)`
is a wrapper for pyplot, I believe imshow but check?

source can be a Band or tuple (dataset, bdx), or array, or dataset in which case the first band is diaplayed unless colorinterp metadata is set - how?

TODO it's really unclear how to plot three bands from a multispectral image! seems possible if you create a band object

WTF is "rasterio band order"?

In [ ]:
from rasterio._